In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic
mp_pose=mp.solutions.pose

# GET VIDEO

In [3]:
vid_cap = cv2.VideoCapture(0)
#add="http://141.215.199.212:8080/video" # This captures video from your phone Ip webcam
#vid_cap.open(add)

# intialize holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while vid_cap.isOpened():
        ret, frame = vid_cap.read()
    # Recoloring our video from BGR to RGB
        img =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        #img.flags.writeable=False
    
    #now time for detections
        results= holistic.process(img)
    
    #Recolor image from RGB to BGR for rendering
        #img.flags.writeable=True
        img =cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        
         #Render pose_connections
        mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
        
        # Face_landmarks
        
        mp_drawing.draw_landmarks(img, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                 mp_drawing.DrawingSpec(color=(80,80,80), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,80,80), thickness=1, circle_radius=1)
                                 )
        # left_hand_connections
        
        mp_drawing.draw_landmarks(img,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
        
        # right_hand_connections
        
        mp_drawing.draw_landmarks(img,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
        # Full screen video
        cv2.namedWindow("POSE CLASSIFICATION", cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty("POSE CLASSIFICATION",cv2.WND_PROP_FULLSCREEN,
               cv2.WINDOW_FULLSCREEN)
    
    
        cv2.imshow('POSE CLASSIFICATION', img)
    
        if cv2.waitKey(10) & 0xFF == ord('v'): # we are comparing the ascii value of LHS and RHS
            break

vid_cap.release()
cv2.destroyAllWindows()

# Making DATA SET 

In [4]:
# Here we will store our data into csv file
import csv
import os

In [5]:
# Add pose landmarks and face into one and store them into an variable
Total_landmarks=len(results.face_landmarks.landmark)+len(results.pose_landmarks.landmark)

In [6]:
Total_landmarks

501

In [27]:
landmarks=['pose'] #initialize variable landmark with 'pose'
for i in range(1,Total_landmarks+1): # range starts from 1 to Total_landmarks+1
    landmarks+=['x{}'.format(i),'y{}'.format(i),'z{}'.format(i),'v{}'.format(i) ] #used .format and for each loop the landmarks is gonna update

<function list.count(value, /)>

In [8]:
# create a csv file
with open('pose_classification.csv',mode='w',newline='') as myfile:
    csv_write= csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_write.writerow(landmarks) # use write row method and write all landmark keys into csv file


# create poses

In [5]:
pose_name='FEMALE WALKING'

In [6]:
vid_cap = cv2.VideoCapture(0)
#add="http://141.215.199.212:8080/video"
#vid_cap.open(add)

# intialize holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while vid_cap.isOpened():
        ret, frame = vid_cap.read()
    # Recoloring our video from BGR to RGB
        img =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        img.flags.writeable=False
    
    #now time for detections
        results= holistic.process(img)
    
    #Recolor image from RGB to BGR for rendering
        img.flags.writeable=True
        img =cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        
         #Render pose_connections
        mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
        
        # Face_landmarks
        
        mp_drawing.draw_landmarks(img, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                 mp_drawing.DrawingSpec(color=(80,80,80), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,80,80), thickness=1, circle_radius=1)
                                 )
        # left_hand_connections
        
        mp_drawing.draw_landmarks(img,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
        
        # right_hand_connections
        
        mp_drawing.draw_landmarks(img,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
        # export landmark cordinates
        # we write our code in try block so that we can handle any errors occured
        try:
            pose=results.pose_landmarks.landmark #store all results in pose variable
            pose_row= list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility]for landmark in pose]).flatten())
            #we used list comprehension we are looping through each landmark in pose and storing the values in array which is named 
            #as pose_row and we flatten the vlaues so higher dimensions will get reduced to single dimension
        
            face=results.face_landmarks.landmark
            face_row= list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility]for landmark in face]).flatten())
            
            # then we will combine pose_row and face_row into one big row
            big_row=pose_row+face_row
            # Append pose_name to big_row
            big_row.insert(0,pose_name)
            # export the data into csv
            
            with open('pose_classification.csv',mode='a',newline='') as myfile:
                csv_write= csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_write.writerow(big_row)
            
        except:
            pass
         # Full screen video
        cv2.namedWindow("POSE CLASSIFICATION", cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty("POSE CLASSIFICATION",cv2.WND_PROP_FULLSCREEN,
               cv2.WINDOW_FULLSCREEN)
    
    
        cv2.imshow('POSE CLASSIFICATION', img)
    
        if cv2.waitKey(10) & 0xFF == ord('v'):
            break
vid_cap.release()
cv2.destroyAllWindows()

# Train Model

In [7]:
# Reading poses completed, Now it's time to train the model.
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
df=pd.read_csv('pose_classification.csv')

In [9]:
df.tail()

,pose,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1421,FEMALE WALKING,0.411650,0.204358,-0.615105,0.999975,0.421246,0.187265,-0.587629,0.999944,0.426582,...,0.001649,0.0,0.431464,0.184212,0.007483,0.0,0.433434,0.182367,0.007749,0.0
1422,FEMALE WALKING,0.413907,0.137652,-0.679856,0.999975,0.423829,0.121328,-0.649525,0.999944,0.430040,...,0.002590,0.0,0.435672,0.110406,0.010490,0.0,0.437737,0.108070,0.010939,0.0
1423,FEMALE WALKING,0.424998,0.092714,-0.672697,0.999975,0.436390,0.073215,-0.638325,0.999945,0.444069,...,0.003318,0.0,0.451814,0.065803,0.011104,0.0,0.453970,0.063449,0.011508,0.0
1424,FEMALE WALKING,0.437818,0.072429,-0.653112,0.999975,0.450462,0.052244,-0.615152,0.999945,0.458894,...,0.004458,0.0,0.462225,0.046308,0.013237,0.0,0.464560,0.043746,0.013734,0.0
1425,FEMALE WALKING,0.460798,0.043034,-0.685612,0.999975,0.474764,0.023039,-0.647377,0.999945,0.483668,...,0.002616,0.0,0.489996,0.005623,0.011292,0.0,0.492683,0.001732,0.011792,0.0


In [10]:
df[df['pose']=='FEMALE WALKING']

,pose,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1182,FEMALE WALKING,0.635422,0.379353,-0.303283,0.999072,0.640489,0.369832,-0.289666,0.997364,0.643598,...,0.000540,0.0,0.647046,0.371062,0.003187,0.0,0.647889,0.369979,0.003258,0.0
1183,FEMALE WALKING,0.635884,0.381100,-0.335409,0.999153,0.641036,0.371626,-0.320672,0.997600,0.644130,...,0.000079,0.0,0.648565,0.370164,0.002963,0.0,0.649383,0.369012,0.003050,0.0
1184,FEMALE WALKING,0.635765,0.381502,-0.384001,0.999231,0.640910,0.371879,-0.368196,0.997824,0.643944,...,0.000029,0.0,0.648131,0.369971,0.003347,0.0,0.648943,0.368993,0.003457,0.0
1185,FEMALE WALKING,0.635763,0.381861,-0.353376,0.999298,0.640861,0.372025,-0.338153,0.998021,0.643859,...,0.000240,0.0,0.647001,0.370465,0.003205,0.0,0.647799,0.369569,0.003301,0.0
1186,FEMALE WALKING,0.635685,0.381565,-0.345134,0.999360,0.640413,0.371969,-0.329741,0.998199,0.643298,...,0.000297,0.0,0.646462,0.370877,0.003584,0.0,0.647258,0.369915,0.003722,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,FEMALE WALKING,0.411650,0.204358,-0.615105,0.999975,0.421246,0.187265,-0.587629,0.999944,0.426582,...,0.001649,0.0,0.431464,0.184212,0.007483,0.0,0.433434,0.182367,0.007749,0.0
1422,FEMALE WALKING,0.413907,0.137652,-0.679856,0.999975,0.423829,0.121328,-0.649525,0.999944,0.430040,...,0.002590,0.0,0.435672,0.110406,0.010490,0.0,0.437737,0.108070,0.010939,0.0
1423,FEMALE WALKING,0.424998,0.092714,-0.672697,0.999975,0.436390,0.073215,-0.638325,0.999945,0.444069,...,0.003318,0.0,0.451814,0.065803,0.011104,0.0,0.453970,0.063449,0.011508,0.0
1424,FEMALE WALKING,0.437818,0.072429,-0.653112,0.999975,0.450462,0.052244,-0.615152,0.999945,0.458894,...,0.004458,0.0,0.462225,0.046308,0.013237,0.0,0.464560,0.043746,0.013734,0.0


In [11]:
cor=df.corr()
print(cor)

            x1        y1        z1        v1        x2        y2        z2  \
x1    1.000000  0.402545 -0.234796 -0.174993  0.993386  0.391026 -0.230743   
y1    0.402545  1.000000 -0.351788 -0.260452  0.434465  0.987139 -0.348192   
z1   -0.234796 -0.351788  1.000000  0.463728 -0.295818 -0.225035  0.998912   
v1   -0.174993 -0.260452  0.463728  1.000000 -0.188819 -0.236753  0.455375   
x2    0.993386  0.434465 -0.295818 -0.188819  1.000000  0.412571 -0.291923   
...        ...       ...       ...       ...       ...       ...       ...   
v500       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
x501  0.967549  0.452271 -0.416225 -0.214491  0.983433  0.409576 -0.411967   
y501  0.395522  0.983895 -0.233120 -0.237075  0.416698  0.995023 -0.228726   
z501  0.217482  0.088201 -0.509962 -0.328342  0.209444  0.048601 -0.487105   
v501       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

            v2        x3        y3  ...      z499  v499      x5

In [12]:
x=df.drop('pose',axis=1)# features
y=df['pose']#target

In [13]:
x

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0.631273,0.628692,-1.350329,0.998967,0.665285,0.567833,-1.284819,0.997817,0.681639,0.568513,...,-0.005595,0.0,0.692570,0.569556,0.013762,0.0,0.698108,0.565421,0.014267,0.0
1,0.633129,0.628914,-1.416636,0.998927,0.666680,0.567869,-1.343579,0.997793,0.682854,0.568541,...,-0.006495,0.0,0.697733,0.572038,0.013416,0.0,0.703246,0.568358,0.013900,0.0
2,0.634044,0.630013,-1.403789,0.998780,0.667477,0.568289,-1.332966,0.997581,0.683475,0.568892,...,-0.007551,0.0,0.698059,0.571863,0.012120,0.0,0.703581,0.568027,0.012656,0.0
3,0.635164,0.630636,-1.371428,0.998647,0.668275,0.568562,-1.295691,0.997435,0.684144,0.569086,...,-0.007651,0.0,0.698966,0.570066,0.011194,0.0,0.704543,0.566060,0.011639,0.0
4,0.635528,0.631006,-1.345409,0.998522,0.668928,0.568625,-1.270121,0.997322,0.684906,0.569130,...,-0.008152,0.0,0.698979,0.570891,0.010779,0.0,0.704520,0.567239,0.011164,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,0.411650,0.204358,-0.615105,0.999975,0.421246,0.187265,-0.587629,0.999944,0.426582,0.188641,...,0.001649,0.0,0.431464,0.184212,0.007483,0.0,0.433434,0.182367,0.007749,0.0
1422,0.413907,0.137652,-0.679856,0.999975,0.423829,0.121328,-0.649525,0.999944,0.430040,0.121914,...,0.002590,0.0,0.435672,0.110406,0.010490,0.0,0.437737,0.108070,0.010939,0.0
1423,0.424998,0.092714,-0.672697,0.999975,0.436390,0.073215,-0.638325,0.999945,0.444069,0.074232,...,0.003318,0.0,0.451814,0.065803,0.011104,0.0,0.453970,0.063449,0.011508,0.0
1424,0.437818,0.072429,-0.653112,0.999975,0.450462,0.052244,-0.615152,0.999945,0.458894,0.053453,...,0.004458,0.0,0.462225,0.046308,0.013237,0.0,0.464560,0.043746,0.013734,0.0


In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,
    random_state=1913)

In [15]:
y_test

139             Victory
37                Happy
1260     FEMALE WALKING
99              Victory
284     Wakanda Forever
             ...       
1334     FEMALE WALKING
1093        MEN WALKING
137             Victory
1113        MEN WALKING
269          NT RAMARAO
Name: pose, Length: 286, dtype: object

In [16]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [17]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
}

In [18]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

C:\Users\harsh\.conda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())])}

In [20]:
fit_models['lr'].predict(x_test)

array(['Victory', 'Happy', 'FEMALE WALKING', 'Victory', 'Wakanda Forever',
       'whistling', 'FEMALE WALKING', 'whistling', 'MEN WALKING',
       'Thinking', 'FEMALE WALKING', 'Wakanda Forever', 'Happy',
       'Thinking', 'MALE WALKING', 'MEN WALKING', 'Super Man',
       'Wakanda Forever', 'Wakanda Forever', 'Wakanda Forever', 'Happy',
       'Happy', 'Victory', 'Victory', 'NT RAMARAO', 'MALE WALKING',
       'FEMALE WALKING', 'Thinking', 'NT RAMARAO', 'Wakanda Forever',
       'Super Man', 'whistling', 'FEMALE WALKING', 'Wakanda Forever',
       'MEN WALKING', 'FEMALE WALKING', 'Thinking', 'MALE WALKING',
       'MALE WALKING', 'MEN WALKING', 'MEN WALKING', 'MEN WALKING',
       'whistling', 'FEMALE WALKING', 'MEN WALKING', 'Thinking',
       'whistling', 'MEN WALKING', 'whistling', 'FEMALE WALKING',
       'Wakanda Forever', 'Victory', 'MEN WALKING', 'MALE WALKING',
       'Wakanda Forever', 'FEMALE WALKING', 'NT RAMARAO', 'Thinking',
       'whistling', 'Victory', 'NT RAMARAO', 

# Make pickle file 

In [21]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 
from playsound import playsound as ps

In [22]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9895104895104895
rc 0.958041958041958


In [23]:
fit_models['lr'].predict(x_test)

array(['Victory', 'Happy', 'FEMALE WALKING', 'Victory', 'Wakanda Forever',
       'whistling', 'FEMALE WALKING', 'whistling', 'MEN WALKING',
       'Thinking', 'FEMALE WALKING', 'Wakanda Forever', 'Happy',
       'Thinking', 'MALE WALKING', 'MEN WALKING', 'Super Man',
       'Wakanda Forever', 'Wakanda Forever', 'Wakanda Forever', 'Happy',
       'Happy', 'Victory', 'Victory', 'NT RAMARAO', 'MALE WALKING',
       'FEMALE WALKING', 'Thinking', 'NT RAMARAO', 'Wakanda Forever',
       'Super Man', 'whistling', 'FEMALE WALKING', 'Wakanda Forever',
       'MEN WALKING', 'FEMALE WALKING', 'Thinking', 'MALE WALKING',
       'MALE WALKING', 'MEN WALKING', 'MEN WALKING', 'MEN WALKING',
       'whistling', 'FEMALE WALKING', 'MEN WALKING', 'Thinking',
       'whistling', 'MEN WALKING', 'whistling', 'FEMALE WALKING',
       'Wakanda Forever', 'Victory', 'MEN WALKING', 'MALE WALKING',
       'Wakanda Forever', 'FEMALE WALKING', 'NT RAMARAO', 'Thinking',
       'whistling', 'Victory', 'NT RAMARAO', 

In [24]:
y_test

139             Victory
37                Happy
1260     FEMALE WALKING
99              Victory
284     Wakanda Forever
             ...       
1334     FEMALE WALKING
1093        MEN WALKING
137             Victory
1113        MEN WALKING
269          NT RAMARAO
Name: pose, Length: 286, dtype: object

In [25]:
with open('body_pose_detect.pkl', 'wb') as file: # write our trained model into pickle file
    pickle.dump(fit_models['lr'], file)

In [26]:
# read pickle file 
with open('body_pose_detect.pkl', 'rb') as file:
    model = pickle.load(file)


In [27]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
vid_cap = cv2.VideoCapture(0)
#add="http://141.215.199.212:8080/video"
#vid_cap.open(add)

# intialize holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while vid_cap.isOpened():
        ret, frame = vid_cap.read()
    # Recoloring our video from BGR to RGB
        img =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        img.flags.writeable=False
    
    #now time for detections
        results= holistic.process(img)
    
    #Recolor image from RGB to BGR for rendering
        img.flags.writeable=True
        img =cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        
         #Render pose_connections
        mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
        
        # Face_landmarks
        
        mp_drawing.draw_landmarks(img, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                 mp_drawing.DrawingSpec(color=(80,80,80), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,80,80), thickness=1, circle_radius=1)
                                 )
        # left_hand_connections
        
        mp_drawing.draw_landmarks(img,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
        
        # right_hand_connections
        
        mp_drawing.draw_landmarks(img,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
        # export landmark cordinates
        # we write our code in try block so that we can handle any errors occured
        try:
            pose=results.pose_landmarks.landmark #store all results in pose variable
            pose_row= list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility]for landmark in pose]).flatten())
            #we used list comprehension we are looping through each landmark in pose and storing the values in array which is named 
            #as pose_row and we flatten the vlaues so higher dimensions will get reduced to single dimension
        
            face=results.face_landmarks.landmark
            face_row= list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility]for landmark in face]).flatten())
            
            # then we will combine pose_row and face_row into one big row
            big_row=pose_row+face_row
            # Append pose_name to big_row
 #           big_row.insert(0,pose_name)
            # export the data into csv
            
 #           with open('pose_classification.csv',mode='a',newline='') as myfile:
  #              csv_write= csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
   #             csv_write.writerow(big_row)
             # Make Detections
            x = pd.DataFrame([big_row])
            body_language_class = model.predict(x)[0]
            body_language_prob = model.predict_proba(x)[0]
            print(body_language_class, body_language_prob)
            
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(img, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(img, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(img, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(img, 'POSE'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(img, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(img, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(img, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        
                
            
            
            
            
        except:
            pass
         # Full screen video
        cv2.namedWindow("POSE CLASSIFICATION", cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty("POSE CLASSIFICATION",cv2.WND_PROP_FULLSCREEN,
               cv2.WINDOW_FULLSCREEN)
    
    
        cv2.imshow('POSE CLASSIFICATION', img)
    
        if cv2.waitKey(10) & 0xFF == ord('v'):
            break
vid_cap.release()
cv2.destroyAllWindows()

C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


whistling [1.15934650e-24 1.62103893e-10 1.17109334e-35 1.56279576e-24
 2.83487334e-30 6.65767775e-25 6.12566764e-25 2.02240077e-10
 1.49474218e-19 1.00000000e+00]


C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


whistling [2.88114404e-24 5.91444253e-11 6.40939242e-36 7.54210893e-26
 6.07438149e-31 3.46300044e-25 1.92032176e-24 1.11470353e-11
 5.80248969e-20 1.00000000e+00]
whistling [4.35612444e-23 8.81152065e-11 3.94657070e-35 1.60698351e-25
 2.56867938e-30 3.68231936e-24 8.09206244e-24 2.66809071e-12
 1.24423674e-19 1.00000000e+00]


C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

FEMALE WALKING [9.99171797e-01 5.65167564e-19 1.65546268e-08 6.33835833e-04
 7.07594532e-08 1.94279515e-04 2.11996665e-10 8.65846679e-23
 2.38928869e-21 3.37770697e-16]


C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


FEMALE WALKING [8.05298110e-01 1.39931600e-19 1.82017637e-07 1.94677973e-01
 5.78377097e-07 2.31550944e-05 2.17866875e-09 4.98049456e-24
 2.48201787e-22 1.07831304e-17]


C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\harsh\.conda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


MEN WALKING [1.00894940e-01 1.11461673e-20 3.17784425e-05 5.08826028e-01
 1.39407219e-04 3.90107827e-01 1.95883844e-08 2.17880718e-23
 9.94664298e-22 8.40995491e-19]
